In [1]:
import json
import os
import pandas as pd
import numpy as np
import glob

In [2]:
#1-60
#61-120
#121-180
#181-240
#241-300
#301-365

In [3]:
df_all = pd.DataFrame()
path = "Json_file/"
all_files = glob.glob(os.path.join(path, "*.json"))
all_files.sort()

In [4]:
def read_json_file(json_name):
    df = []
    for line in open(json_name, 'r'):
        df.append(json.loads(line))
    df = pd.DataFrame(df)
    return df
    

In [13]:
df_all = pd.DataFrame()
count = 1
for i in all_files[330:]:
    df_all = df_all.append(read_json_file(i))
    print(i + "  " + str(count))
    count+=1
clean_first(df_all,'330-365.csv')

Json_file/2022-05-27.json  1
Json_file/2022-05-28.json  2
Json_file/2022-05-29.json  3
Json_file/2022-05-30.json  4
Json_file/2022-05-31.json  5
Json_file/2022-06-01.json  6
Json_file/2022-06-02.json  7
Json_file/2022-06-03.json  8
Json_file/2022-06-04.json  9
Json_file/2022-06-05.json  10
Json_file/2022-06-06.json  11
Json_file/2022-06-07.json  12
Json_file/2022-06-08.json  13
Json_file/2022-06-09.json  14
Json_file/2022-06-10.json  15
Json_file/2022-06-11.json  16
Json_file/2022-06-12.json  17
Json_file/2022-06-13.json  18
Json_file/2022-06-14.json  19
Json_file/2022-06-15.json  20
Json_file/2022-06-16.json  21
Json_file/2022-06-17.json  22
Json_file/2022-06-18.json  23
Json_file/2022-06-19.json  24
Json_file/2022-06-20.json  25
Json_file/2022-06-21.json  26
Json_file/2022-06-22.json  27
Json_file/2022-06-23.json  28
Json_file/2022-06-24.json  29
Json_file/2022-06-25.json  30
Json_file/2022-06-26.json  31
Json_file/2022-06-27.json  32
Json_file/2022-06-28.json  33
Json_file/2022-06-2

In [15]:
path = ""
all_files = glob.glob(os.path.join(path, "*.csv"))
all_files.sort()
all_files

['1-30.csv',
 '120-150.csv',
 '150-180.csv',
 '180-210.csv',
 '210-240.csv',
 '240-270.csv',
 '270-300.csv',
 '300-330.csv',
 '31-60.csv',
 '330-365.csv',
 '60-90.csv',
 '90-120.csv']

In [48]:
df_all = pd.DataFrame()
count = 1
for i in all_files[:6]:
    df_all = df_all.append(pd.read_csv(i))
    print(i + "  " + str(count))
    count+=1

1-30.csv  1
120-150.csv  2
150-180.csv  3
180-210.csv  4
210-240.csv  5
240-270.csv  6


In [49]:
df_all.to_csv('CSV_file/part01.csv')

In [50]:
path = "CSV_file"
all_files = glob.glob(os.path.join(path, "*.csv"))
all_files.sort()
all_files

['CSV_file/part01.csv', 'CSV_file/part02.csv']

In [51]:
df_all = pd.DataFrame()
count = 1
for i in all_files:
    df_all = df_all.append(pd.read_csv(i))
    print(i + "  " + str(count))
    count+=1

CSV_file/part01.csv  1
CSV_file/part02.csv  2


In [52]:
clean_dict(df_all,"CSV_file/Cleaned_all.csv")

actorid
time zone
contained


In [5]:
def to_dict(col):
    index_list = list(range(len(col)))
    res_dct = dict(zip(col, index_list))
    return res_dct

def manual_label(i):
    if i == 'https://coles.hub.clearlrs.com':
        i = "home"
        pass
    if "activities" in i:
        i = "activities"
        pass
    if "course_id" in i:
        i = 'course_id'
        pass
    if "notifications" in i:
        i = 'notifications'
        pass
    if "course/" in i:
        i = "learningseat_course"
        pass
    if "colesplay" in i:
        i = "colesplay"
        pass
    i = i.split('/')[-1] 
    
    return i

def clean_first(df,name):
    df = df.drop(["id"],axis = 1)
    # verb: keep the last word
    list_verb = df['verb'].tolist()
    list_verb = [ i.split('/')[-1] for i in list_verb]
    df['verb'] = list_verb
    print("verb")
    
    # type: keep last word
    list_type = df['type'].tolist()
    list_type = [ str(i).split('/')[-1] for i in list_type]
    for i in range(0,len(list_type)):
        if list_type[i] == 'nan':
            list_type[i] = pd.NA
    df['type'] = list_type
    print("type")

    #Is manager: YN to 10
    df.loc[(df["Is Manager"] == "Y"), "Is Manager"] = 1
    df.loc[(df["Is Manager"] == "N"), "Is Manager"] = 0
    print("manager")
    
    #Is new starter: YN to 10
    df.loc[(df["Is New Starter"] == "Y"), "Is New Starter"] = 1
    df.loc[(df["Is New Starter"] == "N"), "Is New Starter"] = 0
    print("Starter")
    
    # activity: manually label
    list_activity = df['activity']
    list_activity = np.array(list_activity)
    temp_list = []
    for i in list_activity:
        res = manual_label(i)
        temp_list.append(res)

    df['activity'] = temp_list
    print("activity")
    
    # ancestors,[each activities]
    list_ancestors = np.array(df['ancestors'])
    temp_list = []
    for i in list_ancestors:
        lines = str(i)[1:-1].split(",")
        temp_list.append([manual_label(lines[i][14:-1]) for i in range(len(lines)) if i % 2 == 0])
    df['ancestors'] = temp_list
    print("ancestors")
    
    # other: use manaul label
    list_other = np.array(df.other)
    temp_list =[]
    for i in range(len(list_other)):
        if str(list_other[i]) == '<NA>' or str(list_other[i]) == 'nan':
            temp_list.append(pd.NA)
        else:
            temp_list.append(manual_label(str(list_other[i])[15:-15]))
            
    df['other'] = temp_list
    print("other")
    
    df.to_csv(name)
    print("Completed")


In [39]:
def clean_dict(df,name):
    
    # actorid: convert to dict
    list_actorid = df['actorId'].unique()
    dict_actorid = to_dict(list_actorid)
    temp_list = [dict_actorid[i] for i in df['actorId']]
    df['actorId'] = temp_list
    print("actorid")
    
    
    #time zone: convert to dict
    list_timezone = df['Time Zone'].unique()
    dict_timezone = to_dict(list_timezone)
    temp_list = [dict_timezone[i] for i in df['Time Zone']]
    df['Time Zone'] = temp_list
    print("time zone")
    
    
    # contained, [search actorid dict, verb last word]
    list_contain = np.array(df.contained)
    temp_list = []
    for i in range(0,len(list_contain)):
        if str(list_contain[i]) == '<NA>' or str(list_contain[i]) == 'nan':
            temp_list.append(pd.NA)
        else:
            target_id = str(list_contain[i]).split(',')[0][13:-1]
            result = []
            try:
                result.append(dict_actorid[target_id])
            except KeyError:
                result.append('unrecorded')

            contained_verb = str(list_contain[i]).split(',')[1][10:-2].split('/')[-1]
            result.append(contained_verb)
            temp_list.append(result)
    df.contained = temp_list
    df.to_csv(name)
    print("contained")

In [43]:
list_contain = np.array(df_all.contained)
temp_list = []
for i in range(0,len(list_contain)):
        if str(list_contain[i]) == '<NA>' or str(list_contain[i]) == 'nan':
            temp_list.append(pd.NA)
        else:
            print(i)
            break

198


In [44]:
list_contain[198]

[4676, 'shared']